SANDBOX

# Project A

D1.csv  
Knowledge: What categories of products are bought together instead of individually  


- Prepare a transactional dataset where each transaction represents the category of products along with other details.  
- Build an association mining model on this dataset to identify the common product categories that customers purchase.


1. What pre-processing was required on the dataset before building the association mining model? What variables did you include in the analysis? Justify your choice.


2. Conduct association mining and answer the following:
- What ‘min_support’ and `min_confidence’ thresholds were set for this mining exercise? Rationalise why these values were chosen.
- Report and interpret the top-5 rules (as per Lift values).

3. List the five most common product categories that customers bought with the product category ‘01F’.
4. Can you perform sequence analysis on this dataset? If yes, present your results. If not, rationalise why.
5. How can the outcome of this study be used by the relevant decision-makers?


In [1]:
import pandas as pd
from apyori import apriori

In [2]:
# Opening the data file
filename1 = 'data/D1.csv'
filename2 = 'data/D2.csv'
df1 = pd.read_csv(filename1)
df2 = pd.read_csv(filename2)

## Task A.1

In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131706 entries, 0 to 131705
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Date          131706 non-null  object 
 1   Customer_ID   131706 non-null  int64  
 2   Sales_ID      131706 non-null  int64  
 3   SKU_Category  131706 non-null  object 
 4   SKU           131706 non-null  object 
 5   Quantity      131706 non-null  float64
 6   Sales_Amount  131706 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 7.0+ MB


We have 131706 entries and non-null counts shows that there is no missing data. To confirm:

In [4]:
# Checking if we have any null value
df1.isnull().sum()

Date            0
Customer_ID     0
Sales_ID        0
SKU_Category    0
SKU             0
Quantity        0
Sales_Amount    0
dtype: int64

In [5]:
# Checking for duplicate rows
df1.duplicated().sum()

0

In [14]:
df1.head(10)
df1.columns

Index(['Date', 'Customer_ID', 'Sales_ID', 'SKU_Category', 'SKU', 'Quantity',
       'Sales_Amount'],
      dtype='object')

Because the knowlege is "What categories of products are bought together instead of individually", hence we need to focus on the columns that are giving us these information. The rest of them are not relevant.  


The relevant columns are:
- Sales_ID: One purchare holds many SKU
- SKU_Category: As the knowledge mentioned "What categories of products are bought together". Hence our focus would be sku_category rather than sku
- Quantity: Task 3 requires to list 5 most common product category customers bought with the category 01F
- Sales_Amount: this will be useful for decision making on task 5


In [7]:
# Reducing the number of columns to have the necessary ones
df11 = df1[['Date','Sales_ID', 'SKU_Category']]

#lowercasing the feature names for ease
df11.columns = [col.lower() for col in df11.columns]

df11

,date,sales_id,sku_category
0,2/01/2016,1,X52
1,2/01/2016,2,2ML
2,2/01/2016,3,0H2
3,2/01/2016,4,0H2
4,2/01/2016,5,0H2
...,...,...,...
131701,4/07/2016,32900,IEV
131702,4/07/2016,32900,N8U
131703,4/07/2016,32900,U5F
131704,4/07/2016,32900,0H2


## Task A.2

Converting the data to transactional format:  
Each sales_id contains many sku_category. Hence we group categories (sku_category) to per transaction (sales_id)

In [8]:
transactions = df11.groupby(['sales_id'])['sku_category'].apply(list)
transactions

sales_id
1             [X52]
2             [2ML]
3             [0H2]
4             [0H2]
5             [0H2]
            ...    
64678         [YMJ]
64679    [FEW, H15]
64680         [B93]
64681         [P42]
64682         [XG4]
Name: sku_category, Length: 64682, dtype: object

In [11]:
len(transactions)

64682

### Building model
Since we want to check which (at least) two categoris are purchased in one transaction, therefore the chance of each is 50%.  
We check how many categories we have

In [12]:
num_sku_category = len(df11['sku_category'].value_counts())
print('Number of categories:', num_sku_category)

Number of categories: 187


The probeblity of an sku_category to happen is 1/n. n=187  
Our support is between 1 and 0.005 (min support)

In [13]:
min_support = 1/num_sku_category
print('min_support: ', min_support)

min_support:  0.0053475935828877


In [12]:
from apyori import apriori

transaction_list = list(transactions)
results = list(apriori(transaction_list, min_support=min_support))
results[:5]

# from mlxtend.frequent_patterns import apriori, association_rules

## Generate frequent itemsets
#transaction_list = list(transactions)
#frequent_itemsets = apriori(transaction_list, min_support=min_support, use_colnames=True)

## Generate association rules with min_confidence
#rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

## Display the top 5 rules
#print(rules.head())


[RelationRecord(items=frozenset({'01F'}), support=0.02682353668717727, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'01F'}), confidence=0.02682353668717727, lift=1.0)]),
 RelationRecord(items=frozenset({'0H2'}), support=0.06429918679076095, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'0H2'}), confidence=0.06429918679076095, lift=1.0)]),
 RelationRecord(items=frozenset({'0KX'}), support=0.006802510744874927, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'0KX'}), confidence=0.006802510744874927, lift=1.0)]),
 RelationRecord(items=frozenset({'0WT'}), support=0.010296527627469776, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'0WT'}), confidence=0.010296527627469776, lift=1.0)]),
 RelationRecord(items=frozenset({'1EO'}), support=0.013697782999907239, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'1EO'}), confi

In [13]:
# Out of 64682 transactions 142 transactions with min support of ~0.005
print('Number of rules:', len(results))

Number of rules: 142


In [14]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                         rule_set.support, rule.confidence, rule.lift]) 
     
    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 
                                        'Confidence', 'Lift']) 

result_df = convert_apriori_results_to_pandas_df(results)

print(result_df)

    Left_side Right_side   Support  Confidence      Lift
0                    01F  0.026824    0.026824  1.000000
1                    0H2  0.064299    0.064299  1.000000
2                    0KX  0.006803    0.006803  1.000000
3                    0WT  0.010297    0.010297  1.000000
4                    1EO  0.013698    0.013698  1.000000
..        ...        ...       ...         ...       ...
303       N8U    LPF,OXH  0.006973    0.045519  2.268294
304       OXH    LPF,N8U  0.006973    0.167161  9.919540
305   LPF,N8U        OXH  0.006973    0.413761  9.919540
306   LPF,OXH        N8U  0.006973    0.347458  2.268294
307   OXH,N8U        LPF  0.006973    0.626389  8.715011

[308 rows x 5 columns]


The above shows that many items have min support == confidance. These items do not have LHS. This causes the lift to be 1.00, hence these data are not useful and should be removed. However these are still useful to get the frequency of their purchases. For example 0H2 sku_category has a frequency of 0.064 (6.4% times purchased) which is quite high compare to othere categories.

In [15]:
# Filtering out those without Left_side
def filter_out_lhs(df):
    filtered_df = df[(df['Left_side'].notna()) & (df['Left_side'] != '')]
    return filtered_df
    # filtered_df = df[(df['Left_side'].notna())

In [16]:
result_df_cleaned = filter_out_lhs(result_df)
print(result_df_cleaned)

    Left_side Right_side   Support  Confidence      Lift
80        01F        6BZ  0.005612    0.209222  7.460249
81        6BZ        01F  0.005612    0.200110  7.460249
83        01F        FU5  0.007498    0.279539  6.689284
84        FU5        01F  0.007498    0.179430  6.689284
86        01F        IEV  0.012909    0.481268  7.553841
..        ...        ...       ...         ...       ...
303       N8U    LPF,OXH  0.006973    0.045519  2.268294
304       OXH    LPF,N8U  0.006973    0.167161  9.919540
305   LPF,N8U        OXH  0.006973    0.413761  9.919540
306   LPF,OXH        N8U  0.006973    0.347458  2.268294
307   OXH,N8U        LPF  0.006973    0.626389  8.715011

[166 rows x 5 columns]


In [17]:
# Sorting data based on lift (importance)
result_df_cleaned = result_df_cleaned.sort_values(by='Lift', ascending=False)
print(result_df_cleaned)

    Left_side Right_side   Support  Confidence      Lift
305   LPF,N8U        OXH  0.006973    0.413761  9.919540
304       OXH    LPF,N8U  0.006973    0.167161  9.919540
297       OXH    IEV,N8U  0.006076    0.145663  9.507370
298   IEV,N8U        OXH  0.006076    0.396569  9.507370
138       FU5        9ZX  0.007220    0.172771  9.100304
..        ...        ...       ...         ...       ...
104       0H2        N8U  0.009230    0.143544  0.937093
233       N8U        U5F  0.007838    0.051171  0.917867
234       U5F        N8U  0.007838    0.140599  0.917867
231       R6E        N8U  0.008874    0.124620  0.813552
230       N8U        R6E  0.008874    0.057933  0.813552

[166 rows x 5 columns]


**The result for task 2.a**  
What ‘min_support’ and `min_confidence’ thresholds were set for this mining exercise? Rationalise why these values were chosen.

**Min_support:** We have 187 sku_categories. The chance of a category to occure is 1/187.  
**min_confidence:** The maximum confidence is 62%. Even though many rules have confidence less than 50% but they have high value of lift. That says there is still a strong correlation between sku_categories. Therefore we disregarded the min_confidence.

**The result for task 2.b**  
The top five rules are those with the highest lift.

In [18]:
print(result_df_cleaned.head(5))

    Left_side Right_side   Support  Confidence      Lift
305   LPF,N8U        OXH  0.006973    0.413761  9.919540
304       OXH    LPF,N8U  0.006973    0.167161  9.919540
297       OXH    IEV,N8U  0.006076    0.145663  9.507370
298   IEV,N8U        OXH  0.006076    0.396569  9.507370
138       FU5        9ZX  0.007220    0.172771  9.100304


Interpretation:
The above shows that the first five with highest lift.  
1. LPF,N8U => OXH: Customers who purchase categories LPF and N8U are 9.92 times more likely to also purchase OXH.
2. OXH => LPF, N8U: Customers who purchase OXH are 9.92 times more likely to purchase LPF and N8U together despite the low confidence. The high lift shows a strong correlation between purchase of OXH and LPF, N8U.
3. OXH => IEV,N8U: Customers who purchase OXH are 9.51 times more likely to also purchase IEV and N8U.
4.  IEV,N8U => OXH: Customers who purchase IEV and N8U together are 9.51 times more likely to also purchase OXH.
5.  FU5 => 9ZX: Customers who purchase FU5 are 9.10 times more likely to also purchase 9ZX.

## Task A.3
List the five most common product categories that customers bought with the product category '01F'.  

To answer this question we need to check occurence of 01F either in antecedent (LHS) or conceqent (RHS). Then sort it by support as the support tells the frequency.  
We must be careful duplicates. 

In [19]:
# Get all the rows containing '01F' in both Left_side or Right_side
df_01F = result_df_cleaned[(result_df_cleaned['Left_side'].str.contains('01F')) | (result_df_cleaned['Right_side'].str.contains('01F'))]

# Sort by Support to find the most common associations
df_01F = df_01F.sort_values(by='Support', ascending=False)

# Remove duplicates
df_01F = df_01F.groupby(lambda x: frozenset([df_01F.at[x, 'Left_side'].strip(), df_01F.at[x, 'Right_side'].strip()])).first().reset_index(drop=True)

# The top 5 
print(df_01F.head(5))


  Left_side Right_side   Support  Confidence      Lift
0       01F        IEV  0.012909    0.481268  7.553841
1       01F        LPF  0.012198    0.454755  6.327052
2       OXH        01F  0.008163    0.195701  7.295851
3       01F        FU5  0.007498    0.279539  6.689284
4       N8U        01F  0.007421    0.048446  1.806089


**ANSWER**  
{ IEV, LPF, OXH, FU5, N8U }

## Task A.4
Can you perform sequence analysis on this dataset? If yes, present your results. If not, rationalise why.

Note for task ?: If we focus on date and customer_id, then this will be sequencial as it shows at what sequence a customer purchased what category.
Use the jar file for squencial mining. Sort the date first in ascending format. In the result we can see we have order 

In [20]:
df12 = df1[['Date', 'Customer_ID', 'SKU_Category']]

#lowercasing the feature names for ease
df12.columns = [col.lower() for col in df12.columns]

# The date is in format %d/%m/%Y
df12.loc[:, 'date'] = pd.to_datetime(df12['date'], format='%d/%m/%Y').dt.date
df12 = df12.sort_values(by='date')

df12

,date,customer_id,sku_category
0,2016-01-02,2547,X52
156,2016-01-02,3,TW8
157,2016-01-02,427,R6E
158,2016-01-02,427,R6E
159,2016-01-02,427,Q4N
...,...,...,...
128099,2016-12-31,16860,R6E
128100,2016-12-31,16860,R6E
128101,2016-12-31,16860,SFC
128091,2016-12-31,17306,C8Z


In [21]:
sequences = df12.groupby('customer_id')['sku_category'].apply(list).tolist()
sequences[:10]

[['0H2', 'N8U'],
 ['TVL', 'F9B'],
 ['TW8', 'TW8', 'LPF'],
 ['69B', 'YMJ', '29A', 'N8U', 'JR5'],
 ['P42', 'P42', 'P42', 'P42', 'LGI'],
 ['Z23', '1VL', '1EO'],
 ['1VL'],
 ['LPF'],
 ['XG4'],
 ['SJS']]

In [22]:
# Removing single items
sequences = [seq for seq in sequences if len(seq) > 1]
sequences[:5]

[['0H2', 'N8U'],
 ['TVL', 'F9B'],
 ['TW8', 'TW8', 'LPF'],
 ['69B', 'YMJ', '29A', 'N8U', 'JR5'],
 ['P42', 'P42', 'P42', 'P42', 'LGI']]

In [23]:
from collections import defaultdict
import subprocess
import re

''' Uses SPMF to find association rules in supplied transactions '''
def get_association_rules(sequences, min_sup, min_conf):
    # step 1: create required input for SPMF
    
    # prepare a dict to uniquely assign each item in the transactions to an int ID
    item_dict = defaultdict(int)
    output_dict = defaultdict(str)
    item_id = 1
    
    # write your sequences in SPMF format
    with open('seq_rule_input.txt', 'w+') as f:
        for sequence in sequences:
            z = []
            for itemset in sequence:
                # if there are multiple items in one itemset
                if isinstance(itemset, list):
                    for item in itemset:
                        if item not in item_dict:
                            item_dict[item] = item_id
                            item_id += 1

                        z.append(item_dict[item])
                else:
                    if itemset not in item_dict:
                        item_dict[itemset] = item_id
                        output_dict[str(item_id)] = itemset
                        item_id += 1
                    z.append(item_dict[itemset])
                    
                # end of itemset
                z.append(-1)
            
            # end of a sequence
            z.append(-2)
            f.write(' '.join([str(x) for x in z]))
            f.write('\n')
    
    # run SPMF with supplied parameters
    supp_param = '{}%'.format(int(min_sup * 100))
    conf_param = '{}%'.format(int(min_conf * 100))
    subprocess.call(['java', '-jar', 'spmf.jar', 'run', 'RuleGrowth', 
                     'seq_rule_input.txt', 'seq_rule_output.txt', 
                     supp_param, conf_param], shell=True)
    
    # read back the output rules
    outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')
    output_rules = []
    for rule in outputs:
        left, right, sup, conf = re.search(pattern=r'([0-9\,]+) ==> ([0-9\,]+) #SUP: ([0-9]+) #CONF: ([0-9\.]+)', string=rule).groups()
        sup = int(sup) / len(sequences)
        conf = float(conf)
        output_rules.append([[output_dict[x] for x in left.split(',')], [output_dict[x] for x in right.split(',')], sup, conf])
    
    # return pandas DataFrame
    return pd.DataFrame(output_rules, columns = ['Left_rule', 'Right_rule', 'Support', 'Confidence'])

In [24]:
df_13 = get_association_rules(sequences, 0.05, 0.3)
df_13

,Left_rule,Right_rule,Support,Confidence
0,[LPF],[N8U],0.059912,0.366567
1,[IEV],[N8U],0.054766,0.377374
2,[LPF],[IEV],0.063771,0.390180
3,[IEV],[LPF],0.067998,0.468552
4,[LPF],[OXH],0.051703,0.316342


The above does not show any sequence of purhase.

In [26]:
df_14 = df_13.copy()
right_rule_supports = df_13.groupby(df_13['Right_rule'].apply(lambda x: tuple(x)))['Support'].sum().to_dict()
df_14['Lift'] = df_13.apply(lambda row: row['Confidence'] / right_rule_supports[tuple(row['Right_rule'])], axis=1)
df_14

,Left_rule,Right_rule,Support,Confidence,Lift
0,[LPF],[N8U],0.059912,0.366567,3.196493
1,[IEV],[N8U],0.054766,0.377374,3.290737
2,[LPF],[IEV],0.063771,0.390180,6.118441
3,[IEV],[LPF],0.067998,0.468552,6.890671
4,[LPF],[OXH],0.051703,0.316342,6.118441


In [27]:
df_14 = df_14.sort_values(by='Lift', ascending=False)
df_14

,Left_rule,Right_rule,Support,Confidence,Lift
3,[IEV],[LPF],0.067998,0.468552,6.890671
2,[LPF],[IEV],0.063771,0.390180,6.118441
4,[LPF],[OXH],0.051703,0.316342,6.118441
1,[IEV],[N8U],0.054766,0.377374,3.290737
0,[LPF],[N8U],0.059912,0.366567,3.196493


**Analysis:**  
LPF with the confidence range of 18% to 29% is the most important category.  
This category has been purchased with IEV, N8U, FU5, and OXH categories.  
In addition the high lift value of 7.5% for LPF, indicates that the IEV, N8U, FU5, and OXH 7.5 times more likey to be burchased along with IPF cateogry.

## Task A.5
How can the outcome of this study be used by the relevant decision-makers?

Decision makers can be advised on which product categories are most frequently purchased together.
- It also tells which products are correlated
- The result may help on various ways:
  - increase the purchase intake of the famous products
  - reduce the purchase intake of the list famous products
  - boundling products for offer (discount) stragety to increase sale
  - focus on increasing the sale of products that sell less